In [ ]:
#%env MKL_ENABLE_INSTRUCTIONS=SSE4_2

In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from pprint import pprint
import math
import string

stopwords = nltk.corpus.stopwords.words('russian')
# stopwords[:100]
#stopwords = set(["a" ,"the", "'s"])
punctuation_list = set(string.punctuation)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
text = open("/content/NLP.txt").read()
print(text[0:100], "...")

Анализ саундтреков из аккаунтов пользователей позволяет сделать выводы об их музыкальных предпочтени ...


In [10]:
# Sentence Tokenization
sentences = sent_tokenize(text)
print("Sentence Tokenization:")
for idx, sentence in enumerate(sentences[0:5]):
    print(f"{idx+1}: {sentence}")

print("\nWord Tokenization:")
tokenized_text = []
for idx, sentence in enumerate(sentences):
    words = word_tokenize(sentence)
    tokenized_text.append([t.lower() for t in words if t.lower() not in stopwords and t.lower() not in punctuation_list])
    print(f"Sentence {idx+1} Words: {words}")

Sentence Tokenization:
1: Анализ саундтреков из аккаунтов пользователей позволяет сделать выводы об их музыкальных предпочтениях.
2: А используя наработанные коррелянты можно судить о характере пользователя.
3: Например, считается: экстраверты в целом чаще слушают музыку и их музыкальные предпочтения более разнообразны, однако они более склонны к поп, хип-хоп, сценической и современной-хитовой музыке.
4: Экстраверты мало слушают музыку, которая «старше их по возрасту»: т.е.
5: экстраверт 1990 года рождения практически не будет слушать музыку, вышедшую раньше своего рождения.

Word Tokenization:
Sentence 1 Words: ['Анализ', 'саундтреков', 'из', 'аккаунтов', 'пользователей', 'позволяет', 'сделать', 'выводы', 'об', 'их', 'музыкальных', 'предпочтениях', '.']
Sentence 2 Words: ['А', 'используя', 'наработанные', 'коррелянты', 'можно', 'судить', 'о', 'характере', 'пользователя', '.']
Sentence 3 Words: ['Например', ',', 'считается', ':', 'экстраверты', 'в', 'целом', 'чаще', 'слушают', 'музыку'

In [13]:
#tokenized_text
freq_dict  = nltk.FreqDist([t for s in tokenized_text for t in s])
print(len(freq_dict))

160


In [17]:
tokenized_text = [[t if freq_dict[t] > 1 else "UNK" for t in sent] for sent in tokenized_text]
tokenized_text

[['UNK',
  'UNK',
  'UNK',
  'пользователей',
  'UNK',
  'сделать',
  'UNK',
  'музыкальных',
  'предпочтениях'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK',
  'UNK',
  'экстраверты',
  'UNK',
  'чаще',
  'слушают',
  'музыку',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'музыке'],
 ['экстраверты',
  'UNK',
  'слушают',
  'музыку',
  'которая',
  '«',
  'UNK',
  'UNK',
  '»',
  'UNK'],
 ['UNK',
  'UNK',
  'UNK',
  'рождения',
  'практически',
  'слушать',
  'музыку',
  'UNK',
  'UNK',
  'UNK',
  'рождения'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'чаще'],
 ['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK'],
 ['UNK',
  'экстраверты',
  'UNK',
  'UNK',
  'чаще',
  'слушают',
  'музыку',
  'UNK',
  'UNK'],
 ['UNK',
  'практически',
  'слушают',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'слушать',
  'UNK',
  'UNK',
  'UNK',
  'UNK'],
 ['«',
  'доброжелательные',
  '»',
  'люди',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'UNK',
  'песен',
  'предпочит

In [18]:
vocab = set(word for sentence in tokenized_text for word in sentence)
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)
print(vocab_size)

26


In [19]:
# Dataset creation
def make_sequences(tokenized_data, seq_len):
    sequences = []
    for sentence in tokenized_data:
        if len(sentence) >= seq_len + 1:
            for i in range(len(sentence) - seq_len):
                seq = sentence[i:i + seq_len]
                target = sentence[i + seq_len]
                sequences.append((seq, target))
    return sequences

In [21]:
sequences = make_sequences(tokenized_text, seq_len=3)
sequences

[(['UNK', 'UNK', 'UNK'], 'пользователей'),
 (['UNK', 'UNK', 'пользователей'], 'UNK'),
 (['UNK', 'пользователей', 'UNK'], 'сделать'),
 (['пользователей', 'UNK', 'сделать'], 'UNK'),
 (['UNK', 'сделать', 'UNK'], 'музыкальных'),
 (['сделать', 'UNK', 'музыкальных'], 'предпочтениях'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'экстраверты'], 'UNK'),
 (['UNK', 'экстраверты', 'UNK'], 'чаще'),
 (['экстраверты', 'UNK', 'чаще'], 'слушают'),
 (['UNK', 'чаще', 'слушают'], 'музыку'),
 (['чаще', 'слушают', 'музыку'], 'UNK'),
 (['слушают', 'музыку', 'UNK'], 'UNK'),
 (['музыку', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'UNK'),
 (['UNK', 'UNK', 'UNK'], 'музыке'),
 (['экстраверты', 'UNK', 'слушают'], 'музыку'),
 (['UNK', 'слушают', 'музыку'], 'которая'),
 (['слуш

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import random

In [23]:
# Model definition
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded)
        output = self.linear(output[:, -1, :])  # Only last time step
        return output
        #return torch.softmax(output, dim=1)

In [24]:
# Hyperparameters
embedding_dim = 64
hidden_dim = 128
epochs = 500
learning_rate = 0.01
seq_length = 5

model = RNNPredictor(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    random.shuffle(sequences)
    for seq, target in sequences:
        input_tensor = torch.tensor([[word2idx[word] for word in seq]], dtype=torch.long)
        target_tensor = torch.tensor([word2idx[target]], dtype=torch.long)

        optimizer.zero_grad()
        output = model(input_tensor)
        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    if (epoch + 1) % 100 == 0 or epoch == 0:
        torch.save(model.state_dict(), f"./rnn_next_term.ckp{epoch}.pt")

Epoch 1/500, Loss: 344.5248
Epoch 10/500, Loss: 94.5542
Epoch 20/500, Loss: 95.6855
Epoch 30/500, Loss: 94.6750
Epoch 40/500, Loss: 97.3648
Epoch 50/500, Loss: 99.6737
Epoch 60/500, Loss: 99.6214
Epoch 70/500, Loss: 117.8114
Epoch 80/500, Loss: 108.3861
Epoch 90/500, Loss: 101.8351
Epoch 100/500, Loss: 105.0002
Epoch 110/500, Loss: 106.2929
Epoch 120/500, Loss: 106.8030
Epoch 130/500, Loss: 118.7820
Epoch 140/500, Loss: 110.5246
Epoch 150/500, Loss: 113.3353
Epoch 160/500, Loss: 116.3751
Epoch 170/500, Loss: 137.0277
Epoch 180/500, Loss: 114.9475
Epoch 190/500, Loss: 108.6000
Epoch 200/500, Loss: 114.8281
Epoch 210/500, Loss: 121.5153
Epoch 220/500, Loss: 124.1993
Epoch 230/500, Loss: 112.9671
Epoch 240/500, Loss: 119.0923
Epoch 250/500, Loss: 107.7816
Epoch 260/500, Loss: 108.1838
Epoch 270/500, Loss: 114.8276
Epoch 280/500, Loss: 120.0828
Epoch 290/500, Loss: 114.6830
Epoch 300/500, Loss: 115.9799
Epoch 310/500, Loss: 119.7373
Epoch 320/500, Loss: 112.0026
Epoch 330/500, Loss: 121.23

In [25]:
torch.save(model.state_dict(), "./rnn_next_term.pt")

In [26]:
rnn = RNNPredictor(vocab_size, embedding_dim, hidden_dim)
rnn.load_state_dict(torch.load("./rnn_next_term.pt"))
rnn.eval()

RNNPredictor(
  (embedding): Embedding(26, 64)
  (gru): GRU(64, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=26, bias=True)
)

In [27]:
def predict_next_word(model, input_words):
    input_tensor = torch.tensor([[word2idx[word] for word in input_words]], dtype=torch.long)
    predicted_idx= model(input_tensor).softmax(dim=1).argmax(dim=1).item()
    return idx2word[predicted_idx]

In [32]:
test_seq = "экстраверты".split(" ")
print(test_seq)
for i in range(10):
    predicted = predict_next_word(rnn, test_seq)
    test_seq.append(predicted)
    print(test_seq)

['экстраверты']
['экстраверты', 'чаще']
['экстраверты', 'чаще', 'слушают']
['экстраверты', 'чаще', 'слушают', 'музыку']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK', 'UNK', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']
['экстраверты', 'чаще', 'слушают', 'музыку', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK']
